# MapLibre live updates: incremental marker moves + coffee stop

This notebook demonstrates a **live / incremental** update pattern suitable for streaming coordinates (e.g., MQTT):

- the marker is moved in-place via `move_marker(...)` (no redraw loop)
- a tiny state machine pauses when the walker reaches a coffee shop

In [ ]:
import importlib

import simulated_city.maplibre_live as maplibre_live
from IPython.display import display

importlib.reload(maplibre_live)
LiveMapLibreMap = maplibre_live.LiveMapLibreMap

CITY_HALL_LNGLAT = (12.5683, 55.6761)

m = LiveMapLibreMap(center=CITY_HALL_LNGLAT, zoom=16.5, height="650px")
display(m)

In [ ]:
from __future__ import annotations

import asyncio
import math
import random
import time
from typing import Iterable, Tuple

LngLat = Tuple[float, float]

# A few approximate coffee-shop locations near City Hall.
COFFEE_SHOPS: list[LngLat] = [
    (12.5699, 55.6763),
    (12.5669, 55.6758),
    (12.5689, 55.6749),
]


def haversine_m(a: LngLat, b: LngLat) -> float:
    """Great-circle distance in meters between two (lng, lat) points."""
    lng1, lat1 = a
    lng2, lat2 = b
    r = 6_371_000.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = phi2 - phi1
    dlambda = math.radians(lng2 - lng1)
    h = (math.sin(dphi / 2) ** 2) + math.cos(phi1) * math.cos(phi2) * (math.sin(dlambda / 2) ** 2)
    return 2 * r * math.asin(math.sqrt(h))


def is_near_any(point: LngLat, targets: Iterable[LngLat], *, radius_m: float) -> bool:
    return any(haversine_m(point, t) <= radius_m for t in targets)


def nearest_index_within(point: LngLat, targets: list[LngLat], *, radius_m: float) -> int | None:
    best_idx: int | None = None
    best_d = float("inf")
    for idx, t in enumerate(targets):
        d = haversine_m(point, t)
        if d <= radius_m and d < best_d:
            best_d = d
            best_idx = idx
    return best_idx


def set_marker(
    map_widget: LiveMapLibreMap,
    *,
    marker_id: str,
    lnglat: LngLat,
    color: str,
    popup: str | None = None,
    remove_first: bool = True,
) -> None:
    # `move_marker` does not update marker color in-place, so we force a refresh.
    if remove_first:
        try:
            map_widget.remove_marker(marker_id)
        except Exception:
            pass
    lng, lat = lnglat
    map_widget.add_marker(lng, lat, name=marker_id, color=color, popup=popup)


def add_static_coffee_shop_markers(map_widget: LiveMapLibreMap) -> None:
    # Use fixed IDs; remove first so re-running the cell doesn't stack markers.
    for idx, (lng, lat) in enumerate(COFFEE_SHOPS, start=1):
        set_marker(
            map_widget,
            marker_id=f"coffee-{idx}",
            lnglat=(lng, lat),
            color="#2e7d32",
            popup=f"Coffee shop {idx}",
            remove_first=True,
        )


async def live_random_walk(
    map_widget: LiveMapLibreMap,
    *,
    marker_id: str = "walker",
    seed: int = 42,
    step_m: float = 6.0,
    step_s: float = 0.5,
    max_radius_m: float = 250.0,
    coffee_enter_radius_m: float = 50.0,  # within 50m -> goes in
    coffee_stop_s: float = 5.0,  # set to 300.0 for 5 minutes
    n_steps: int = 400,
) -> None:
    rng = random.Random(seed)
    center_lng, center_lat = CITY_HALL_LNGLAT
    meters_per_deg_lat = 111_320.0
    meters_per_deg_lng = 111_320.0 * math.cos(math.radians(center_lat))

    # Offsets in meters from the center.
    x_m = 0.0
    y_m = 0.0

    map_widget.move_marker(marker_id, CITY_HALL_LNGLAT)

    visited_shops: set[int] = set()
    stopped_until: float | None = None
    for _ in range(n_steps):
        now = time.monotonic()
        if stopped_until is not None and now < stopped_until:
            await asyncio.sleep(min(step_s, stopped_until - now))
            continue
        stopped_until = None

        theta = rng.random() * 2.0 * math.pi
        x_m += step_m * math.cos(theta)
        y_m += step_m * math.sin(theta)

        # Soft-bounds.
        r = math.hypot(x_m, y_m)
        if r > max_radius_m:
            scale = max_radius_m / r
            x_m *= scale
            y_m *= scale

        lng = center_lng + (x_m / meters_per_deg_lng)
        lat = center_lat + (y_m / meters_per_deg_lat)
        pos = (lng, lat)

        # Move marker incrementally (live-update style).
        map_widget.move_marker(marker_id, pos)

        # Coffee-stop rule: within 50m -> goes in and waits.
        shop_idx = nearest_index_within(pos, COFFEE_SHOPS, radius_m=coffee_enter_radius_m)
        if shop_idx is not None:
            stopped_until = time.monotonic() + coffee_stop_s
            if shop_idx not in visited_shops:
                visited_shops.add(shop_idx)
                shop_id = f"coffee-{shop_idx + 1}"
                set_marker(
                    map_widget,
                    marker_id=shop_id,
                    lnglat=COFFEE_SHOPS[shop_idx],
                    color="#fdd835",
                    popup=f"Coffee shop {shop_idx + 1} (visited)",
                    remove_first=True,
                )

        await asyncio.sleep(step_s)


add_static_coffee_shop_markers(m)

# Ensure the moving marker exists (makes it obvious the JS method works).
m.move_marker("walker", CITY_HALL_LNGLAT, color="#ff0000")

# Start the simulation as a background task.
task = asyncio.create_task(live_random_walk(m))
print("Simulation started in background. Run Cell 5 to stop.")

# --- MQTT wiring sketch (optional) ---
# In production you would call `m.move_marker("walker", (lng, lat))` from
# a safe consumer loop fed by MQTT messages (avoid calling into widgets from
# a background thread directly).

In [ ]:
import asyncio

# Wait a moment so the background task can enqueue updates.
await asyncio.sleep(2)

print("queued js calls:", len(m._js_calls))
if m._js_calls:
    print("last method:", m._js_calls[-1].get("method"))

print("move_marker_supported:", getattr(m, "_move_marker_supported", None))
print("move_marker_acks:", getattr(m, "_move_marker_ack_count", None))

In [ ]:
# Stop the background task.
task.cancel()